# Road Safety Data Warehousing and Analytics  
**ETL, Star Schema Design, SQL Analytics & Association Rule Mining**

This notebook presents a data warehousing and analytics pipeline built on
real-world road safety datasets.

The project focuses on:
- integrating heterogeneous raw data sources using Python-based ETL,
- designing a dimensional data warehouse (star schema),
- performing analytical SQL queries for business-style insights,
- and applying association rule mining to discover meaningful patterns
  in road fatality data.

The workflow reflects a typical analytical data engineering process,
from raw data preparation to structured storage and downstream analysis.

### Academic Context

This work was completed as a **group project (2 members)** for  
**CITS5504 – Data Warehousing**, University of Western Australia.

**Contributors**:  
- Boya Zhang  
- Lyu Lu  

**Date**: April 2025

## 1. Setup

**Importing the libraries**

In [1]:
# Import libraries for data manipulation and analysis
import pandas as pd
import numpy as np

# Import modules for association rule mining
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

**Loading the datasets**

In [2]:
# Load fatality data
fatalities = pd.read_excel("./DataResource/bitre_fatalities_dec2024.xlsx", 
                           sheet_name="BITRE_Fatality",
                           skiprows=4,
                           header=0
                          )

# Load crash data
crashes = pd.read_excel("./DataResource/bitre_fatal_crashes_dec2024.xlsx", 
                           sheet_name="BITRE_Fatal_Crash",
                           skiprows=4,
                           header=0
                          )

# Load population LGA data
population_LGA = pd.read_excel(("./DataResource/"
                           "Population estimates by LGA, Significant Urban Area, "
                           "Remoteness Area and electoral division, "
                           "2001 to 2024.xlsx"),
                           sheet_name="Table 1",
                           skiprows=5,
                           header=None
                          )

# Load dwelling data
dwelling = pd.read_csv("./DataResource/LGA (count of dwellings).csv", 
                       skiprows=10, 
                       header=0, 
                       index_col=False
                      )

# Load population state data
population_state = pd.read_excel("./DataResource/population_state.xlsx",
                           sheet_name="state",
                           header=0
                          )

## 2. Data Exploration

**Data structure and content**

In [3]:
# Display shapes of datasets
print("Fatalities shape:", fatalities.shape)
print("Crashes shape:", crashes.shape)
print("Population LGA shape:", population_LGA.shape)
print("Population State shape:", population_state.shape)
print("Dwelling shape:", dwelling.shape)

Fatalities shape: (56874, 23)
Crashes shape: (51284, 23)
Population LGA shape: (552, 26)
Population State shape: (44, 9)
Dwelling shape: (561, 2)


In [4]:
# Top few rows
fatalities.head()

,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Bus Involvement,Heavy Rigid Truck Involvement,Articulated Truck Involvement,...,Age,National Remoteness Areas,SA4 Name 2021,National LGA Name 2021,National Road Type,Christmas Period,Easter Period,Age Group,Day of week,Time of day
0,20241115,NSW,12,2024,Friday,04:00:00,Single,No,No,No,...,74,Inner Regional Australia,Riverina,Wagga Wagga,Arterial Road,Yes,No,65_to_74,Weekday,Night
1,20241125,NSW,12,2024,Friday,06:15:00,Single,No,No,No,...,19,Inner Regional Australia,Sydney - Baulkham Hills and Hawkesbury,Hawkesbury,Local Road,No,No,17_to_25,Weekday,Day
2,20246013,Tas,12,2024,Friday,09:43:00,Multiple,No,No,No,...,33,Inner Regional Australia,Launceston and North East,Northern Midlands,Local Road,Yes,No,26_to_39,Weekday,Day
3,20241002,NSW,12,2024,Friday,10:35:00,Multiple,No,No,No,...,32,Outer Regional Australia,New England and North West,Armidale Regional,National or State Highway,No,No,26_to_39,Weekday,Day
4,20242261,Vic,12,2024,Friday,11:30:00,Multiple,-9,-9,-9,...,62,Unknown,NaN,NaN,Undetermined,No,No,40_to_64,Weekday,Day


In [5]:
crashes.head()

,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Number Fatalities,Bus \nInvolvement,Heavy Rigid Truck Involvement,...,SA4 Name 2021,National LGA Name 2021,National Road Type,Christmas Period,Easter Period,Day of week,Time of Day,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,20241115,NSW,12,2024,Friday,04:00:00,Single,1,No,No,...,Riverina,Wagga Wagga,Arterial Road,Yes,No,Weekday,Night,NaN,NaN,NaN
1,20241125,NSW,12,2024,Friday,06:15:00,Single,1,No,No,...,Sydney - Baulkham Hills and Hawkesbury,Hawkesbury,Local Road,No,No,Weekday,Day,NaN,NaN,NaN
2,20246013,Tas,12,2024,Friday,09:43:00,Multiple,1,No,No,...,Launceston and North East,Northern Midlands,Local Road,Yes,No,Weekday,Day,NaN,NaN,NaN
3,20241002,NSW,12,2024,Friday,10:35:00,Multiple,1,No,No,...,New England and North West,Armidale Regional,National or State Highway,No,No,Weekday,Day,NaN,NaN,NaN
4,20242261,Vic,12,2024,Friday,11:30:00,Multiple,1,-9,-9,...,NaN,NaN,Undetermined,No,No,Weekday,Day,NaN,NaN,NaN


In [6]:
population_LGA.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,NaN,NaN,2001,2002,2003,2004,2005,2006,2007,2008,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
1,LGA code,LGA name (a),no.,no.,no.,no.,no.,no.,no.,no.,...,no.,no.,no.,no.,no.,no.,no.,no.,no.,no.
2,10050,Albury,45265,45816,46180,46505,47004,47566,48140,48518,...,51486,52171,53056,53922,54657,55466,56067,56665,57509,58317
3,10180,Armidale,27906,27774,27610,27410,27350,27377,27468,27788,...,29160,29310,29519,29631,29701,29600,29332,29361,29588,29646
4,10250,Ballina,37856,38417,38870,39120,39305,39537,39824,40020,...,42336,42993,43652,44385,44997,45663,46196,46849,47277,47935


In [7]:
population_state.head()

,Unnamed: 0,NSW,Vic,Qld,SA,WA,Tas,NT,ACT
0,1981,5249455,3957333,2367477,1321235,1311284,427925,125186,228782
1,1982,5315846,4002731,2442912,1334090,1348096,430308,131517,233778
2,1983,5363744,4045185,2493373,1349553,1375244,433909,137942,240055
3,1984,5416536,4086549,2535976,1362611,1397817,438866,143934,246259
4,1985,5478254,4129796,2583368,1373324,1427370,443548,150596,253446


In [8]:
dwelling.head()

,LGA (EN),Unnamed: 1
0,Albury,25430
1,Armidale Regional,12955
2,Ballina,20889
3,Balranald,1091
4,Bathurst Regional,18458


In [9]:
# Data types, non-null counts
fatalities.info()
crashes.info()
population_state.info()
population_LGA.info()
dwelling.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56874 entries, 0 to 56873
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Crash ID                       56874 non-null  int64 
 1   State                          56874 non-null  object
 2   Month                          56874 non-null  int64 
 3   Year                           56874 non-null  int64 
 4   Dayweek                        56874 non-null  object
 5   Time                           56831 non-null  object
 6   Crash Type                     56874 non-null  object
 7   Bus Involvement                56874 non-null  object
 8   Heavy Rigid Truck Involvement  56874 non-null  object
 9   Articulated Truck Involvement  56874 non-null  object
 10  Speed Limit                    56874 non-null  object
 11  Road User                      56874 non-null  object
 12  Gender                         56874 non-null  object
 13  A

**Notes**

- The fatalities and crash datasets contain similar information. Only the fatalities dataset will be used for further processing, as it includes all relevant individual-level details for each fatality.
- LGA name fields differ across datasets and will be as key fields for joining
- Some LGA fields are missing and will remain as NaN — rows will be kept to preserve historical fatality records.
- Population data is in wide format and will be reshaped; dwelling data requires column renaming.

## 3. Data Cleaning and Preparation

**Data processing**

In [10]:
population_LGA.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,NaN,NaN,2001,2002,2003,2004,2005,2006,2007,2008,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
1,LGA code,LGA name (a),no.,no.,no.,no.,no.,no.,no.,no.,...,no.,no.,no.,no.,no.,no.,no.,no.,no.,no.
2,10050,Albury,45265,45816,46180,46505,47004,47566,48140,48518,...,51486,52171,53056,53922,54657,55466,56067,56665,57509,58317
3,10180,Armidale,27906,27774,27610,27410,27350,27377,27468,27788,...,29160,29310,29519,29631,29701,29600,29332,29361,29588,29646
4,10250,Ballina,37856,38417,38870,39120,39305,39537,39824,40020,...,42336,42993,43652,44385,44997,45663,46196,46849,47277,47935


In [11]:
# Clean population_LGA data
population_LGA.iloc[0, :2] = population_LGA.iloc[1, :2]
population_LGA.columns = population_LGA.iloc[0]
population_LGA = population_LGA.drop(index=[0, 1]).reset_index(drop=True)

# Reshape population_LGA data from wide to long format
population_LGA = population_LGA.melt(
    id_vars=["LGA code", "LGA name (a)"],
    var_name="Year",
    value_name="Population_LGA"
)

# drop rows where LGA name or Year is missing
population_LGA = population_LGA.dropna(subset=["LGA name (a)", "Year"])

In [12]:
population_state.head()

,Unnamed: 0,NSW,Vic,Qld,SA,WA,Tas,NT,ACT
0,1981,5249455,3957333,2367477,1321235,1311284,427925,125186,228782
1,1982,5315846,4002731,2442912,1334090,1348096,430308,131517,233778
2,1983,5363744,4045185,2493373,1349553,1375244,433909,137942,240055
3,1984,5416536,4086549,2535976,1362611,1397817,438866,143934,246259
4,1985,5478254,4129796,2583368,1373324,1427370,443548,150596,253446


In [13]:
# Transformation
population_state = population_state.rename(columns={population_state.columns[0]:"Year"})
population_state_long = population_state.melt(
  id_vars="Year",
  var_name= "State",
  value_name="Population_State"
)
population_state_long["Year"] = pd.to_numeric(population_state_long["Year"], errors="coerce")
population_state_long["Population_State"] = pd.to_numeric(population_state_long["Population_State"], errors="coerce")

In [ ]:
# population_state = population_state_long
# population_state.head()

,Year,State,Population_State
0,1981,NSW,5249455
1,1982,NSW,5315846
2,1983,NSW,5363744
3,1984,NSW,5416536
4,1985,NSW,5478254


**Data merging**

In [15]:
population_LGA.head()

,LGA code,LGA name (a),Year,Population_LGA
0,10050,Albury,2001,45265
1,10180,Armidale,2001,27906
2,10250,Ballina,2001,37856
3,10300,Balranald,2001,2751
4,10470,Bathurst,2001,35504


In [16]:
# Merge dwellings with population_LGA LGA
dwelling.columns = ["LGA name (a)", "Dwelling"]
population_LGA = population_LGA.merge(dwelling, on="LGA name (a)", how="left")

# Convert data types to numeric types
population_LGA["Population_LGA"] = pd.to_numeric(population_LGA["Population_LGA"],errors="coerce")
population_LGA["Dwelling"] = pd.to_numeric(population_LGA["Dwelling"], errors="coerce")
population_LGA = population_LGA.drop(columns=["LGA code"])

population_LGA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13152 entries, 0 to 13151
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   LGA name (a)    13152 non-null  object 
 1   Year            13152 non-null  object 
 2   Population_LGA  13152 non-null  int64  
 3   Dwelling        12720 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 411.1+ KB


In [17]:
# Merge population LGA and dwelling info into fatalities
fatalities = fatalities.merge(
    population_LGA, left_on=["National LGA Name 2021", "Year"],
    right_on=["LGA name (a)", "Year"],
    how="left"
)

# Merge population LGA and dwelling info into fatalities
fatalities = fatalities.merge(
    population_state, left_on=["State", "Year"],
    right_on=["State", "Year"],
    how="left"
)

fatalities.shape

(56874, 27)

### Missing values

In [18]:
# Replace unknown invalid or missing codes with NaN
fatalities = fatalities.replace(
    [-9, "-9", "unknown", "Unknown", "Other/-9", "Undetermined"],
    np.nan
)
# whether we can just replace these values to "unknown"

/var/folders/j6/bx5wzhgd4lxc5fnmfg3s5nvw0000gn/T/ipykernel_77350/82292955.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fatalities = fatalities.replace(


- populationis reshaped from wide to long format and merged with dwelling.
- fatalities is merged with population and dwelling based on LGA and year.
- Non-standard values are converted as NaN. These rows are kept in the dataset to preserve historical records.

## 4. Dimension Tables

**Location Dimension**

In [19]:
# Create location dimension

location_df = fatalities[["National LGA Name 2021", "State"]].copy()
location_df["National LGA Name 2021"] = location_df["National LGA Name 2021"].fillna("Unknown")  # Create "Unknown" category
location_df = location_df.drop_duplicates()
location_df.insert(0, "LocationID", range(1, 1 + len(location_df)))
location_df.head()

,LocationID,National LGA Name 2021,State
0,1,Wagga Wagga,NSW
1,2,Hawkesbury,NSW
2,3,Northern Midlands,Tas
3,4,Armidale Regional,NSW
4,5,Unknown,Vic


- Extracts unique LGA and State combinations.
- Fill LGA with "Unknown" category.
- LocationID as primary key.

**Date Dimension**

In [20]:
# Create date dimension
date_df = fatalities[["Month", "Year"]].drop_duplicates().dropna()

# Add Quarter column
date_df["Quarter"] = ((date_df["Month"] - 1) // 3) + 1

date_df.insert(0, "DateID", range(1, 1 + len(date_df)))
date_df.head()

,DateID,Month,Year,Quarter
0,1,12,2024,4
113,2,11,2024,4
244,3,10,2024,4
359,4,9,2024,3
454,5,8,2024,3


In [21]:
date_df["Year"].unique()

array([2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014,
       2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003,
       2002, 2001, 2000, 1999, 1998, 1997, 1996, 1995, 1994, 1993, 1992,
       1991, 1990, 1989])

- Extracts unique Year, Month, and Dayweek combinations.
- Adds Quarter column.
- DateID as primary key.

**Dayweek Dimension**

In [22]:
# Create dayweek_df
dayweek_df = fatalities[["Dayweek"]].drop_duplicates().dropna()
dayweek_df.insert(0, "DayweekID", range(1, 1 + len(dayweek_df)))
dayweek_df

,DayweekID,Dayweek
0,1,Friday
14,2,Monday
31,3,Saturday
56,4,Sunday
83,5,Thursday
92,6,Tuesday
105,7,Wednesday


**Date of week Dimension**

In [23]:
# Create day_of_week_df and 

day_of_week_df = fatalities[["Day of week"]].drop_duplicates().dropna()
day_of_week_df.insert(0, "DayofWeekID", range(1, 1 + len(day_of_week_df)))
day_of_week_df

,DayofWeekID,Day of week
0,1,Weekday
8,2,Weekend


We noticed that some “Dayweek” values were misclassified. Since the “Day of week” feature is not useful for further analysis, we decided not to correct them.

**Holiday Dimension**

In [24]:
# Create holiday status column in fatalities
fatalities["Holiday Status"] = np.where(
    (fatalities["Christmas Period"] == "Yes") | 
    (fatalities["Easter Period"] == "Yes"), "Holiday", "Non-holiday")

In [25]:
# Create holiday dimension
holiday_df = fatalities[["Holiday Status"]].drop_duplicates().dropna()
holiday_df.insert(0, "HolidayID", range(1, 1 + len(holiday_df)))
holiday_df

,HolidayID,Holiday Status
0,1,Holiday
1,2,Non-holiday


- Creates Holiday Status in fatalities based on Christmas Period and Easter Period.
- Extracts unique Holiday Status values.
- HolidayID as primary key.

**Time Dimension**

In [26]:
# Create time dimension
time_df = fatalities[["Time of day"]].drop_duplicates().dropna()
time_df.insert(0, "TimeID", range(1, 1 + len(time_df)))
time_df

,TimeID,Time of day
0,1,Night
1,2,Day


- Extracts unique Time of day values: Day and Night.
- TimeID as primary key.

**Crash Type Dimension**

In [27]:
# Create crash type dimension
crashtype_df = fatalities[["Crash Type"]].drop_duplicates().dropna()
crashtype_df.insert(0, "CrashTypeID", range(1, 1 + len(crashtype_df)))
crashtype_df

,CrashTypeID,Crash Type
0,1,Single
2,2,Multiple


- Extracts unique Crash Type values: Single and Multiple.
- CrashTypeID as primary key.

**Involvement Dimension**

In [28]:
# Create involvement columns in fatalities and fill missing values
fatalities[["Bus Involvement", "Heavy Rigid Truck Involvement", "Articulated Truck Involvement"]] = (
    fatalities[["Bus Involvement", "Heavy Rigid Truck Involvement", "Articulated Truck Involvement"]]
    .fillna("No")
)

# Create Involvement Type column based on involvement values
fatalities["Involvement Type"] = np.select(
    [
        (fatalities["Bus Involvement"] == "Yes") & (fatalities["Heavy Rigid Truck Involvement"] == "No") 
            & (fatalities["Articulated Truck Involvement"] == "No"),
        (fatalities["Bus Involvement"] == "No") & (fatalities["Heavy Rigid Truck Involvement"] == "Yes") 
            & (fatalities["Articulated Truck Involvement"] == "No"),
        (fatalities["Bus Involvement"] == "No") & (fatalities["Heavy Rigid Truck Involvement"] == "No") 
            & (fatalities["Articulated Truck Involvement"] == "Yes"),
        (fatalities["Bus Involvement"] == "Yes") & (fatalities["Heavy Rigid Truck Involvement"] == "Yes") 
            & (fatalities["Articulated Truck Involvement"] == "No"),  # Add Bus/Heavy Rigid
        (fatalities["Bus Involvement"] == "Yes") & (fatalities["Heavy Rigid Truck Involvement"] == "No") 
            & (fatalities["Articulated Truck Involvement"] == "Yes"),
        (fatalities["Bus Involvement"] == "No") & (fatalities["Heavy Rigid Truck Involvement"] == "Yes") 
            & (fatalities["Articulated Truck Involvement"] == "Yes"),
    ], 
    ["Bus", "Heavy Rigid Truck", "Articulated Truck", "Bus/Heavy Rigid", "Bus/Articulated Truck", "Heavy Rigid/Articulated Truck"],
    default=None
)

In [29]:
# Create involvement dimension
involvement_df = fatalities[["Involvement Type"]].drop_duplicates().dropna()
involvement_df.insert(0, "InvolvementID", range(1, 1 + len(involvement_df)))
involvement_df

,InvolvementID,Involvement Type
13,1,Articulated Truck
32,2,Bus/Articulated Truck
41,3,Heavy Rigid Truck
102,4,Heavy Rigid/Articulated Truck
103,5,Bus
547,6,Bus/Heavy Rigid


- Creates Involvement Type in fatalities, classifying combinations like Bus, Heavy Rigid Truck, Articulated Truck, and their combinations.
- Includes all 6 types: Bus, Heavy Rigid Truck, Articulated Truck, Bus/Heavy Rigid, Bus/Articulated Truck, and Heavy Rigid/Articulated Truck.
- InvolvementID as primary key.

**User Dimension**

In [30]:
# Create user dimension
user_df = fatalities[["Road User"]].drop_duplicates().dropna()
user_df.insert(0, "UserID", range(1, 1 + len(user_df)))
user_df

,UserID,Road User
0,1,Driver
4,2,Passenger
6,3,Motorcycle rider
12,4,Pedestrian
50,5,Pedal cyclist
488,6,Motorcycle pillion passenger


- Extracts unique Road User values.
- UserID as primary key.

**Road Dimension**

In [31]:
# Create road dimension
road_df = fatalities[["National Road Type"]].drop_duplicates().dropna()
road_df.insert(0, "RoadID", range(1, 1 + len(road_df)))
road_df

,RoadID,National Road Type
0,1,Arterial Road
1,2,Local Road
3,3,National or State Highway
6,4,Sub-arterial Road
17,5,Collector Road
154,6,Pedestrian Thoroughfare
283,7,Access road
970,8,Busway


- Extracts unique National Road Type values.
- RoadID as primary key.

**Speed Limit Dimension**

In [32]:
# Create speed limit categories in fatalities

# Very Low : 0-30
# Low: 31-60
# Medium: 61-90
# High: 91-110
# Very high: >110

fatalities["Speed Limit Category"] = np.select(
    [(fatalities["Speed Limit"] == 5) | (fatalities["Speed Limit"] == 10) | 
        (fatalities["Speed Limit"] == 15)| (fatalities["Speed Limit"] == 20) | 
        (fatalities["Speed Limit"] == 25) | (fatalities["Speed Limit"] == 30),  # Very low
    (fatalities["Speed Limit"] == '<40') | (fatalities["Speed Limit"] == 40) | (fatalities["Speed Limit"] == 50) 
        | (fatalities["Speed Limit"] == 60),  # Low
    (fatalities["Speed Limit"] == 70) | (fatalities["Speed Limit"] == 75) | (fatalities["Speed Limit"] == 80) 
        | (fatalities["Speed Limit"] == 90),  # Medium
    (fatalities["Speed Limit"] == 100) | (fatalities["Speed Limit"] == 110),  # High
    (fatalities["Speed Limit"] == 130)  # Very High
    ], ["Very Low", "Low", "Medium", "High", "Very High"], default=None)

In [33]:
# Create speed limit dimension
speed_limit_df = fatalities[["Speed Limit Category"]].drop_duplicates().dropna()
speed_limit_df.insert(0, "SpeedID", range(1, 1 + len(speed_limit_df)))
speed_limit_df

,SpeedID,Speed Limit Category
0,1,High
1,2,Medium
2,3,Low
63,4,Very Low
554,5,Very High


- Extracts unique Speed Limit Category values from fatalities.
- Categories: Very Low, Low, Medium, High, and Very High.
- SpeedID as primary key.

**Gender Dimension**

In [34]:
# Create gender dimension
gender_df = fatalities[["Gender"]].drop_duplicates().dropna()
gender_df.insert(0, "GenderID", range(1, 1 + len(gender_df)))
gender_df

,GenderID,Gender
0,1,Male
1,2,Female


- Extracts unique `Gender` values: `Male` and `Female`.
- `GenderID` as primary key.

**Age Group Dimension**

In [35]:
# Create age group dimension
age_group_df = fatalities[["Age Group"]].drop_duplicates().dropna()
age_group_df.insert(0, "AgeGroupID", range(1, 1 + len(age_group_df)))
age_group_df

,AgeGroupID,Age Group
0,1,65_to_74
1,2,17_to_25
2,3,26_to_39
4,4,40_to_64
18,5,75_or_older
20,6,0_to_16


- Extracts unique Age Group values from fatalities.
- AgeGroupID as primary key.

## 5. Fact Table 

In [36]:
# Create fact table
fact_df = fatalities.copy()
print(fact_df.columns.tolist())

['Crash ID', 'State', 'Month', 'Year', 'Dayweek', 'Time', 'Crash Type', 'Bus Involvement', 'Heavy Rigid Truck Involvement', 'Articulated Truck Involvement', 'Speed Limit', 'Road User', 'Gender', 'Age', 'National Remoteness Areas', 'SA4 Name 2021', 'National LGA Name 2021', 'National Road Type', 'Christmas Period', 'Easter Period', 'Age Group', 'Day of week', 'Time of day', 'LGA name (a)', 'Population_LGA', 'Dwelling', 'Population_State', 'Holiday Status', 'Involvement Type', 'Speed Limit Category']


**Merge with dimension tables**

In [37]:
# Merge with date dimension
fact_df = fact_df.merge(date_df, on = ["Month", "Year"], how="left")

# Merge with location dimension
fact_df["National LGA Name 2021"] = fact_df["National LGA Name 2021"].fillna("Unknown")
fact_df = fact_df.merge(location_df, on = ["National LGA Name 2021", "State"], how="left")

dfs = [dayweek_df, day_of_week_df, holiday_df, time_df, crashtype_df, user_df, road_df, gender_df, age_group_df, 
       speed_limit_df, involvement_df]
for df in dfs:
    fact_df = fact_df.merge(df, on = df.columns[1], how="left")

# Select columns
fact_df = fact_df[["DateID", "LocationID",  "DayweekID",  "DayofWeekID", "HolidayID", "TimeID", "CrashTypeID", "UserID", "RoadID", 
                   "GenderID", "AgeGroupID", "SpeedID", "InvolvementID", "Population_LGA", "Population_State", "Dwelling"]]

In [38]:
# Create measures
fact_df["Fatality"] = 1
fact_df["Fatality Rate per Population LGA"] = (fact_df["Fatality"] / 
                                           fact_df["Population_LGA"]).where(fact_df["Population_LGA"].notna())*100000
fact_df["Fatality Rate per Population State"] = (fact_df["Fatality"] / 
                                           fact_df["Population_State"]).where(fact_df["Population_State"].notna())*100000
fact_df["Fatality Rate per Dwelling"] = (fact_df["Fatality"] / 
                                         fact_df["Dwelling"]).where(fact_df["Dwelling"].notna())*100000


fact_df.insert(0, "FatalitiesID", range(1, 1+len(fact_df)))

# Convert columns data_type
for col in fact_df.columns:
    if col not in ["Fatality Rate per Population LGA", "Fatality Rate per Population State", "Fatality Rate per Dwelling"]:
        fact_df[col] = fact_df[col].astype('Int64') 

# Drop the features population and dwelling, as these features cannot be additive
fact_df = fact_df.drop(["Population_LGA","Population_State","Dwelling"],axis=1)
fact_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56874 entries, 0 to 56873
Data columns (total 18 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   FatalitiesID                        56874 non-null  Int64  
 1   DateID                              56874 non-null  Int64  
 2   LocationID                          56874 non-null  Int64  
 3   DayweekID                           56874 non-null  Int64  
 4   DayofWeekID                         56861 non-null  Int64  
 5   HolidayID                           56874 non-null  Int64  
 6   TimeID                              56830 non-null  Int64  
 7   CrashTypeID                         56874 non-null  Int64  
 8   UserID                              56742 non-null  Int64  
 9   RoadID                              10738 non-null  Int64  
 10  GenderID                            56840 non-null  Int64  
 11  AgeGroupID                          56757

- Copy fatalities and merge with dimension tables.
- Calculate measures: Fatality, Fatality Rate per Population LGA, Fatality Rate per Population State, Fatality Rate per Dwelling.
- FatalitiesID as the primary key.

In [39]:
# Export dimension and fact tables to CSV
csv_names = [
    "DimDayweek.csv", "DimDayofWeek.csv", "DimHoliday.csv", "DimTime.csv", "DimCrashType.csv", 
    "DimUser.csv", "DimRoad.csv", "DimGender.csv", "DimAgeGroup.csv", "DimSpeedLimit.csv", 
    "DimInvolvement.csv","DimDate.csv", "DimLocation.csv", "FactFatalities.csv"
]

dfs.extend([date_df, location_df, fact_df])
for i in range(len(dfs)):
    dfs[i].to_csv(f"./DataWarehouse/{csv_names[i]}", index=False)

## 6. Associate Rule Mining

In [40]:
# Load data
df_raw = fatalities.copy()

**Data Preparation**

In [41]:
df_raw.shape

(56874, 30)

In [42]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56874 entries, 0 to 56873
Data columns (total 30 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Crash ID                       56874 non-null  int64  
 1   State                          56874 non-null  object 
 2   Month                          56874 non-null  int64  
 3   Year                           56874 non-null  int64  
 4   Dayweek                        56874 non-null  object 
 5   Time                           56831 non-null  object 
 6   Crash Type                     56874 non-null  object 
 7   Bus Involvement                56874 non-null  object 
 8   Heavy Rigid Truck Involvement  56874 non-null  object 
 9   Articulated Truck Involvement  56874 non-null  object 
 10  Speed Limit                    55389 non-null  object 
 11  Road User                      56742 non-null  object 
 12  Gender                         56840 non-null 

In [43]:
df_raw.head()

,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Bus Involvement,Heavy Rigid Truck Involvement,Articulated Truck Involvement,...,Age Group,Day of week,Time of day,LGA name (a),Population_LGA,Dwelling,Population_State,Holiday Status,Involvement Type,Speed Limit Category
0,20241115,NSW,12,2024,Friday,04:00:00,Single,No,No,No,...,65_to_74,Weekday,Night,Wagga Wagga,68951.0,28244.0,8511151,Holiday,None,High
1,20241125,NSW,12,2024,Friday,06:15:00,Single,No,No,No,...,17_to_25,Weekday,Day,Hawkesbury,68704.0,25523.0,8511151,Non-holiday,None,Medium
2,20246013,Tas,12,2024,Friday,09:43:00,Multiple,No,No,No,...,26_to_39,Weekday,Day,Northern Midlands,14428.0,6444.0,575959,Holiday,None,Low
3,20241002,NSW,12,2024,Friday,10:35:00,Multiple,No,No,No,...,26_to_39,Weekday,Day,NaN,NaN,NaN,8511151,Non-holiday,None,High
4,20242261,Vic,12,2024,Friday,11:30:00,Multiple,No,No,No,...,40_to_64,Weekday,Day,NaN,NaN,NaN,7012962,Non-holiday,None,None


**Features selection**

In [ ]:
# Based on the main risk factors
features = ['Dayweek', 'Road User', 'Gender', 'National Remoteness Areas','National LGA Name 2021', 
            'National Road Type',  'Age Group', 'Day of week', 'Time of day', 'Involvement Type',
            'Speed Limit Category']

In [46]:
df_select = df_raw[features]
df_select.head()

,Dayweek,Road User,Gender,National Remoteness Areas,National LGA Name 2021,National Road Type,Age Group,Day of week,Time of day,Involvement Type,Speed Limit Category
0,Friday,Driver,Male,Inner Regional Australia,Wagga Wagga,Arterial Road,65_to_74,Weekday,Night,None,High
1,Friday,Driver,Female,Inner Regional Australia,Hawkesbury,Local Road,17_to_25,Weekday,Day,None,Medium
2,Friday,Driver,Female,Inner Regional Australia,Northern Midlands,Local Road,26_to_39,Weekday,Day,None,Low
3,Friday,Driver,Female,Outer Regional Australia,Armidale Regional,National or State Highway,26_to_39,Weekday,Day,None,High
4,Friday,Passenger,Male,NaN,NaN,NaN,40_to_64,Weekday,Day,None,None


In [47]:
# Missing value
df_select.isna().sum().sort_values(ascending=False)

Involvement Type             48492
National Road Type           46136
National LGA Name 2021       45849
National Remoteness Areas    45520
Speed Limit Category          1485
Road User                      132
Age Group                      117
Time of day                     44
Gender                          34
Day of week                     13
Dayweek                          0
dtype: int64

In [48]:
# Replace the nan with Unknow
df_select = df_select.fillna("Unknow")
df_select.isna().sum()

Dayweek                      0
Road User                    0
Gender                       0
National Remoteness Areas    0
National LGA Name 2021       0
National Road Type           0
Age Group                    0
Day of week                  0
Time of day                  0
Involvement Type             0
Speed Limit Category         0
dtype: int64

In [49]:
# Data label
for x,i in enumerate(features):
  df_select.iloc[:,x] = i + ":" + df_select.iloc[:,x].astype(str)

In [50]:
df_select.head()

,Dayweek,Road User,Gender,National Remoteness Areas,National LGA Name 2021,National Road Type,Age Group,Day of week,Time of day,Involvement Type,Speed Limit Category
0,Dayweek:Friday,Road User:Driver,Gender:Male,National Remoteness Areas:Inner Regional Austr...,National LGA Name 2021:Wagga Wagga,National Road Type:Arterial Road,Age Group:65_to_74,Day of week:Weekday,Time of day:Night,Involvement Type:Unknow,Speed Limit Category:High
1,Dayweek:Friday,Road User:Driver,Gender:Female,National Remoteness Areas:Inner Regional Austr...,National LGA Name 2021:Hawkesbury,National Road Type:Local Road,Age Group:17_to_25,Day of week:Weekday,Time of day:Day,Involvement Type:Unknow,Speed Limit Category:Medium
2,Dayweek:Friday,Road User:Driver,Gender:Female,National Remoteness Areas:Inner Regional Austr...,National LGA Name 2021:Northern Midlands,National Road Type:Local Road,Age Group:26_to_39,Day of week:Weekday,Time of day:Day,Involvement Type:Unknow,Speed Limit Category:Low
3,Dayweek:Friday,Road User:Driver,Gender:Female,National Remoteness Areas:Outer Regional Austr...,National LGA Name 2021:Armidale Regional,National Road Type:National or State Highway,Age Group:26_to_39,Day of week:Weekday,Time of day:Day,Involvement Type:Unknow,Speed Limit Category:High
4,Dayweek:Friday,Road User:Passenger,Gender:Male,National Remoteness Areas:Unknow,National LGA Name 2021:Unknow,National Road Type:Unknow,Age Group:40_to_64,Day of week:Weekday,Time of day:Day,Involvement Type:Unknow,Speed Limit Category:Unknow


In [51]:
df_select.shape

(56874, 11)

### Unknown values

In [52]:
# TransactionEncoder() function only can handle string type
df_new = df_select.astype(str)

# TransactionEncoder() was designed to covert lists to array
list =df_new.values.tolist()

# Covert the list to one-hot encoded boolean numpy array. 
# Apriori function allows boolean data type only, such as 1 and 0, or FALSE and TRUE.
te = TransactionEncoder()
array_te = te.fit(list).transform(list)

# Check the array
array_te

# Check the colunms
te.columns_

# Apriori function can handle dataframe only, covert the array to a dataframe
df_arm = pd.DataFrame(array_te, columns = te.columns_)

In [53]:
df_arm.head()

,Age Group:0_to_16,Age Group:17_to_25,Age Group:26_to_39,Age Group:40_to_64,Age Group:65_to_74,Age Group:75_or_older,Age Group:Unknow,Day of week:Unknow,Day of week:Weekday,Day of week:Weekend,...,Road User:Unknow,Speed Limit Category:High,Speed Limit Category:Low,Speed Limit Category:Medium,Speed Limit Category:Unknow,Speed Limit Category:Very High,Speed Limit Category:Very Low,Time of day:Day,Time of day:Night,Time of day:Unknow
0,False,False,False,False,True,False,False,False,True,False,...,False,True,False,False,False,False,False,False,True,False
1,False,True,False,False,False,False,False,False,True,False,...,False,False,False,True,False,False,False,True,False,False
2,False,False,True,False,False,False,False,False,True,False,...,False,False,True,False,False,False,False,True,False,False
3,False,False,True,False,False,False,False,False,True,False,...,False,True,False,False,False,False,False,True,False,False
4,False,False,False,True,False,False,False,False,True,False,...,False,False,False,False,True,False,False,True,False,False


In [54]:
#Find the frequent itemsets
frequent_itemsets = apriori(df_arm,min_support=0.2,use_colnames =True)

#Check the length of rules
frequent_itemsets['length']=frequent_itemsets['itemsets'].apply(lambda x: len(x))

#Assume the length is 2 and the min support is >= 0.3
frequent_itemsets[ (frequent_itemsets['length']==2) & 
                  (frequent_itemsets['support']>=0.3)]

,support,itemsets,length
22,0.411242,"(Day of week:Weekday, Gender:Male)",2
23,0.477037,"(Involvement Type:Unknow, Day of week:Weekday)",2
24,0.471164,"(Day of week:Weekday, National LGA Name 2021:U...",2
25,0.467912,"(Day of week:Weekday, National Remoteness Area...",2
26,0.474329,"(Day of week:Weekday, National Road Type:Unknow)",2
29,0.395541,"(Day of week:Weekday, Time of day:Day)",2
30,0.306537,"(Gender:Male, Day of week:Weekend)",2
31,0.375356,"(Involvement Type:Unknow, Day of week:Weekend)",2
32,0.334758,"(National LGA Name 2021:Unknow, Day of week:We...",2
33,0.332226,"(National Remoteness Areas:Unknow, Day of week...",2


In [55]:
#Assume the min confidence is 0.5
rules_con = association_rules(frequent_itemsets, metric="confidence",min_threshold=0.5)

In [56]:
#Assume the min lift is 1
rules_lift = association_rules(frequent_itemsets, metric="lift",min_threshold=1)

#Based on min confidence (=0.5), 
#output antecedents, consequents, support, confidence and lift.
result_arm = rules_con[['antecedents','consequents','support','confidence','lift']]

In [57]:
#Find the rules whose confidence >= 0.5
new_result_arm = result_arm[result_arm['confidence']>=0.5]

ranked_result = new_result_arm.sort_values(by=['lift', 'confidence'], ascending=False)

ranked_result

,antecedents,consequents,support,confidence,lift
1927,"(Speed Limit Category:High, National LGA Name ...","(Road User:Driver, National Road Type:Unknow, ...",0.204030,0.551468,1.544807
1926,"(Road User:Driver, National Road Type:Unknow, ...","(Speed Limit Category:High, National LGA Name ...",0.204030,0.571541,1.544807
1174,"(Speed Limit Category:High, National LGA Name ...","(Road User:Driver, National Remoteness Areas:U...",0.204311,0.552229,1.544807
1171,"(Road User:Driver, National Remoteness Areas:U...","(Speed Limit Category:High, National LGA Name ...",0.204311,0.571541,1.544807
1932,"(Road User:Driver, National Remoteness Areas:U...","(National Road Type:Unknow, Speed Limit Catego...",0.204030,0.570754,1.544368
...,...,...,...,...,...
850,"(Involvement Type:Unknow, Gender:Male, Nationa...",(Time of day:Day),0.246123,0.504360,0.885529
1617,"(Involvement Type:Unknow, National Remoteness ...",(Time of day:Day),0.246123,0.504360,0.885529
1737,"(Involvement Type:Unknow, National Road Type:U...",(Time of day:Day),0.245807,0.504311,0.885444
2170,"(National LGA Name 2021:Unknow, Involvement Ty...",(Time of day:Day),0.245807,0.504311,0.885444


### Without unknown values

In [58]:
#Load data
df_acc = df_raw[features]

In [59]:
# Missing value
df_acc.isna().sum().sort_values(ascending=False)

Involvement Type             48492
National Road Type           46136
National LGA Name 2021       45849
National Remoteness Areas    45520
Speed Limit Category          1485
Road User                      132
Age Group                      117
Time of day                     44
Gender                          34
Day of week                     13
Dayweek                          0
dtype: int64

In [60]:
# Drop features
# Considering the missing data and the uncontrollable factors 
df_acc = df_acc.drop(columns=["Involvement Type","National Road Type","National LGA Name 2021","National Remoteness Areas"])

In [61]:
# Date Cleaning
df_acc = df_acc.dropna()
df_acc.shape

(55140, 7)

In [62]:
# Data label
df_acc.iloc[:, 1] = "Road User: " + df_acc.iloc[:, 1].astype(str)
df_acc.iloc[:, 3] = "Age: " + df_acc.iloc[:, 3].astype(str)
df_acc.iloc[:, 6] = "Speed: " + df_acc.iloc[:, 6].astype(str)

In [63]:
df_acc.head()

,Dayweek,Road User,Gender,Age Group,Day of week,Time of day,Speed Limit Category
0,Friday,Road User: Driver,Male,Age: 65_to_74,Weekday,Night,Speed: High
1,Friday,Road User: Driver,Female,Age: 17_to_25,Weekday,Day,Speed: Medium
2,Friday,Road User: Driver,Female,Age: 26_to_39,Weekday,Day,Speed: Low
3,Friday,Road User: Driver,Female,Age: 26_to_39,Weekday,Day,Speed: High
5,Friday,Road User: Passenger,Female,Age: 40_to_64,Weekday,Day,Speed: High


In [64]:
# TransactionEncoder() function only can handle string type
df_new1 = df_acc.astype(str)

# TransactionEncoder() was designed to covert lists to array
list =df_new1.values.tolist()

# Covert the list to one-hot encoded boolean numpy array. 
# Apriori function allows boolean data type only, such as 1 and 0, or FALSE and TRUE.
te = TransactionEncoder()
array_te = te.fit(list).transform(list)

# Check the array
array_te

# Check the colunms
te.columns_

# Apriori function can handle dataframe only, covert the array to a dataframe
df_arm = pd.DataFrame(array_te, columns = te.columns_)

In [65]:
df_arm.head()

,Age: 0_to_16,Age: 17_to_25,Age: 26_to_39,Age: 40_to_64,Age: 65_to_74,Age: 75_or_older,Day,Female,Friday,Male,...,Speed: Low,Speed: Medium,Speed: Very High,Speed: Very Low,Sunday,Thursday,Tuesday,Wednesday,Weekday,Weekend
0,False,False,False,False,True,False,False,False,True,True,...,False,False,False,False,False,False,False,False,True,False
1,False,True,False,False,False,False,True,True,True,False,...,False,True,False,False,False,False,False,False,True,False
2,False,False,True,False,False,False,True,True,True,False,...,True,False,False,False,False,False,False,False,True,False
3,False,False,True,False,False,False,True,True,True,False,...,False,False,False,False,False,False,False,False,True,False
4,False,False,False,True,False,False,True,True,True,False,...,False,False,False,False,False,False,False,False,True,False


In [66]:
#Find the frequent itemsets
frequent_itemsets = apriori(df_arm,min_support=0.2,use_colnames =True)

#Check the length of rules
frequent_itemsets['length']=frequent_itemsets['itemsets'].apply(lambda x: len(x))

#Assume the length is 2 and the min support is >= 0.3
frequent_itemsets[ (frequent_itemsets['length']==2) & 
                  (frequent_itemsets['support']>=0.3)]

,support,itemsets,length
13,0.382082,"(Male, Day)",2
16,0.397153,"(Weekday, Day)",2
17,0.336616,"(Night, Male)",2
18,0.342202,"(Road User: Driver, Male)",2
19,0.332680,"(Speed: High, Male)",2
21,0.412586,"(Weekday, Male)",2
22,0.306112,"(Male, Weekend)",2


In [67]:
#Assume the min confidence is 0.5
rules_con = association_rules(frequent_itemsets, metric="confidence",min_threshold=0.5)

In [68]:
#Assume the min lift is 1
rules_lift = association_rules(frequent_itemsets, metric="lift",min_threshold=1)

#Based on min confidence (=0.5), 
#output antecedents, consequents, support, confidence and lift.
result_arm = rules_con[['antecedents','consequents','support','confidence','lift']]

In [69]:
#Find the rules whose confidence >= 0.5
new_result_arm = result_arm[result_arm['confidence']>=0.5]

ranked_result = new_result_arm.sort_values(by=['lift', 'confidence'], ascending=False)

ranked_result

,antecedents,consequents,support,confidence,lift
14,(Night),(Weekend),0.236162,0.550055,1.342690
15,(Weekend),(Night),0.236162,0.576475,1.342690
26,"(Male, Speed: High)",(Road User: Driver),0.201106,0.604503,1.329145
25,"(Road User: Driver, Male)",(Speed: High),0.201106,0.587684,1.252410
16,(Road User: Driver),(Speed: High),0.266522,0.586012,1.248848
17,(Speed: High),(Road User: Driver),0.266522,0.567983,1.248848
6,(Day),(Weekday),0.397153,0.695958,1.178922
5,(Weekday),(Day),0.397153,0.672760,1.178922
23,"(Male, Day)",(Weekday),0.262223,0.686301,1.162565
21,"(Weekday, Male)",(Day),0.262223,0.635560,1.113736
